In [68]:
from models import OutcomeModel
from MIMIC3 import MIMIC_3
from eICU import eICU
from utilities import *
from Processing import *
from graph_builders.diagnoses import DiagnosesGraph as Graph

import networkx as nx

In [69]:
#Get input
#Check results

In [70]:
mimic_path =  "/notebooks/Experiments/Datasets/MIMIC 3/"
dataset = MIMIC_3(mimic_path,save_steps = True)

In [71]:
eicu_path ="/notebooks/Experiments/Datasets/eICU/"
dataset = eICU(eicu_path)

In [74]:
%%time
df = dataset.get_label().sample(2500)


CPU times: user 22.9 s, sys: 1.03 s, total: 23.9 s
Wall time: 28.7 s


In [75]:
df.mortality.sum()

102

In [76]:
df

,diagnoses,mortality
uniquepid,,
015-94201,"[[4280, 4289, 4280, 4289, 79902, 51881], [4280...",0
006-238186,"[[1983, 1983, 4329, 4329, 4329, 4329, 4329, 43...",0
016-7138,"[[29181, 29181, 29181, 29181, 29181, 29181, 29...",0
006-151795,"[[51881, 51881, 0389, 0389, 0389, 0389, 51881,...",0
031-11590,"[[5856, 5856, 5789, 5789, 99592, 99592, 2859, ...",0
...,...,...
021-145624,"[[486, 486, 51881, 51881, 436, 436], [51882, 4...",0
006-115297,"[[42731, 42731, 42781, 42781, 42781, 42781, 42...",0
006-137144,"[[4280, 4289, 4280, 4289, 2869, 2867, 2869, 28...",0


In [77]:
dataset = df.sample(int(df.shape[0]*0.25))
tasks = ['mortality']

x = dataset.diagnoses.values
y ={}
for task in tasks:
    dim = len(np.array(list(dataset[task])).shape)
    y[task] = np.array(list(dataset[task]))
    if task=='mortality':
        y[task] = y[task].reshape(-1,1)


In [78]:
G = nx.DiGraph()
f_name = "CCS-MultiDiagnosisGrouper.txt"

G = get_or_create_diagnosis_ccs_graph(G,f_name,connected=True)


This should never happen.. V173


In [79]:
device = 'cpu'
embedding_size = 50
n_layer = 0

In [81]:
#LOAD MODEL
model = OutcomeModel(embedding_size = embedding_size,n_layers=n_layer,output_pathway = [1])

tasks = ['mortality']
#file = f"models/mimic/['diagnoses']/{tasks}/normal/its:1/dropout/ccs/[283,1]/25/256/SW-Phenotype-{n_layer}-{embedding_size}-causal:False"
#model.load_state_dict(torch.load(file,map_location='cpu'))


In [82]:
%%time
embeddings = init_embeddings(embedding_size,poincare = True,G=G) 


CPU times: user 96 ms, sys: 3.98 ms, total: 100 ms
Wall time: 51 ms


In [83]:
%%time
validation = get_or_load_data(patients = x,target=y,G = G,embedding_size = embedding_size,graph_type=Graph,embeddings=embeddings,replicate=False,random=False).to(device)

CPU times: user 38.6 s, sys: 276 ms, total: 38.9 s
Wall time: 13.7 s


In [84]:
validation = validation.to(device)

In [85]:
validation.target['mortality'].sum()

tensor(27.)

In [89]:
validation.target['mortality'].shape

torch.Size([625, 1])

In [87]:
out = model(validation)

In [88]:
validate(out,validation.target)

{'mortality': {'sensivity': 1.0, 'tp': 27, 'fn': 0, 'fp': 598, 'tn': 0, 'specificity': 0.0, 'died': 27.0, 'accuracy': 0.0432}}


{'mortality': {'sensivity': 1.0,
  'tp': 27,
  'fn': 0,
  'fp': 598,
  'tn': 0,
  'specificity': 0.0,
  'died': 27.0,
  'accuracy': 0.0432}}

In [55]:
def fnrate(y_pred,y_real):
    y_pred = y_pred.cpu().detach().numpy()
    y_real = y_real.cpu().detach().numpy()
    c_matrix = confusion_matrix(y_real, y_pred.round(),labels=[0,1])
    tn, fp, fn, tp  =c_matrix.ravel()
    #return {"False Negative Rate":fn/(tp+fn),
    return {"sensivity":    tp / (tp + fn), "tp":tp,"fn":fn,"fp":fp,"tn":tn,"specificity":tn/(tn+fp)}

In [58]:
y_pred = out[0]
y_real = validation.target['mortality']

In [59]:
fnrate(y_pred,y_real)

{'sensivity': 0.9977011494252873,
 'tp': 434,
 'fn': 1,
 'fp': 11314,
 'tn': 1,
 'specificity': 8.837825894829872e-05}

In [65]:
y_pred.shape

torch.Size([11750, 1])

In [66]:
y_real.shape

torch.Size([11750, 1])

In [26]:
(validation.target['mortality'] == out[0].round()).sum()


tensor(10162)

In [33]:
((validation.target['mortality'] == 1) & (out[0].round()==1)).sum()

tensor(0)